In [1]:
import pandas as pd

from typing import Optional, Union
import pandas as pd, numpy as np, torch
from dataclasses import dataclass
from transformers import AutoTokenizer, AutoConfig
from transformers import EarlyStoppingCallback
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, RobertaForMultipleChoice
import argparse
from transformers import get_polynomial_decay_schedule_with_warmup, TrainerCallback
import datasets
from datasets import Dataset

In [4]:
df_train = pd.read_csv("dataset/random_train.csv")

In [ ]:
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

In [ ]:
df_train.loc[:, 'prompt'] = df_train['prompt'].apply(process)
df_train.loc[:, 'response_a'] = df_train['response_a'].apply(process)
df_train.loc[:, 'response_b'] = df_train['response_b'].apply(process)

In [ ]:
t = df_train.loc[df_train.index % 4 != 0,].reset_index(drop = True)

In [ ]:
t.loc[20,:]

In [ ]:
def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    return label[-1]
    
df_train = pd.read_csv('dataset/train.csv').reset_index(drop = True)
df_train['label'] = df_train.apply(lambda x: get_label(x), axis = 1)

In [ ]:
df_train.loc[df_train.index % 4 != 0,]

In [ ]:
df_train.loc[df_train.index % 20 == 0,].reset_index(drop = True).to_csv('dataset/random_valid.csv')
df_train.loc[df_train.index % 20 != 0,].reset_index(drop = True).to_csv('dataset/random_train.csv')

In [ ]:
df_train.loc[df_train.index % 20 == 0,]

# For instruction

In [3]:
def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    if label[-1] == 0:
        return 'A'
    elif label[-1] == 1:
        return 'B'
    else:
        return 'C'
    return label[-1]
    
train = pd.read_csv('dataset/train.csv').reset_index(drop = True)
train['label'] = train.apply(lambda x: get_label(x), axis = 1)

In [5]:
from tqdm import tqdm
output_as = []
output_bs = []
scuess_idx = []
labels = []
ids = []
for idx, row in tqdm(train.iterrows(), total=len(train)):
    try:
        prompt = row["prompt"]
        response_a = row["response_a"]
        response_b = row["response_b"]
        label = row['label']
        id = row['id']
        if prompt.find('","') != -1:
            cnt = prompt.count('","')
            prompt = row["prompt"].split('","')
            response_a = row["response_a"].split('","')
            response_b = row["response_b"].split('","')
            for sentence_idx in range(cnt + 1):
                output_a = "###Model A\n"
                output_b = "###Model B\n"
                output_a += "Prompt: " + prompt[sentence_idx][2:].strip() + "\n"
                output_a += "Response: " + response_a[sentence_idx][2:].strip() + "\n\n"
                output_b += "Prompt: " + prompt[sentence_idx][2:].strip() + "\n"
                output_b += "Response: " +response_b[sentence_idx][2:].strip() + "\n\n"
                output_as.append(output_a)
                output_bs.append(output_b)
                scuess_idx.append(idx)
                labels.append(label)
                ids.append(id)
        else:
            output_a = "###Model A\nPrompt: " + row["prompt"][2:-2].strip() + "\nResponse: " + row["response_a"][2:-2].strip() + "\n\n"
            output_b = "###Model B\nPrompt: " + row["prompt"][2:-2].strip() + "\nResponse: " + row["response_b"][2:-2].strip() + "\n\n"
            output_as.append(output_a)
            output_bs.append(output_b)
            scuess_idx.append(idx)
            labels.append(label)
            ids.append(id)
        
    except Exception as e:
        print(f"Error: {idx}")
        print(f"{e}")
        # output_as.append(" ")
        # output_bs.append(" ")

  5%|▍         | 2602/57477 [00:00<00:04, 13191.03it/s]

Error: 207
list index out of range
Error: 385
list index out of range
Error: 491
list index out of range
Error: 763
list index out of range
Error: 1763
list index out of range
Error: 2530
list index out of range
Error: 2648
list index out of range
Error: 2770
list index out of range
Error: 2989
list index out of range
Error: 3152
list index out of range


  9%|▉         | 5382/57477 [00:00<00:03, 13663.15it/s]

Error: 3774
list index out of range
Error: 3951
list index out of range
Error: 4165
list index out of range
Error: 4197
list index out of range
Error: 4348
list index out of range
Error: 5390
list index out of range
Error: 6593
list index out of range


 17%|█▋        | 9667/57477 [00:00<00:03, 14144.34it/s]

...

Error: 49295
list index out of range
Error: 49401
list index out of range
Error: 50374
list index out of range
Error: 51240
list index out of range
Error: 51463
list index out of range
Error: 51664
list index out of range
Error: 51836
list index out of range
Error: 52149
list index out of range


 95%|█████████▍| 54497/57477 [00:03<00:00, 14180.07it/s]

Error: 52680
list index out of range
Error: 53194
list index out of range
Error: 53281
list index out of range
Error: 54087
list index out of range
Error: 54096
list index out of range
Error: 54765
list index out of range
Error: 54931
list index out of range
Error: 54952
list index out of range
Error: 55385
list index out of range


100%|██████████| 57477/57477 [00:04<00:00, 14191.76it/s]

Error: 55856
list index out of range
Error: 55912
list index out of range
Error: 56737
list index out of range
Error: 56743
list index out of range


In [6]:
train = pd.DataFrame({'id': ids, 'instruction_a': output_as, 'instruction_b': output_bs, 'label': labels })

In [7]:
train

,id,instruction_a,instruction_b,label
0,30192,###Model A\nPrompt: Is it morally right to try...,###Model B\nPrompt: Is it morally right to try...,A
1,30192,"###Model A\nPrompt: , does pineapple belong on...","###Model B\nPrompt: , does pineapple belong on...",A
2,53567,###Model A\nPrompt: What is the difference bet...,###Model B\nPrompt: What is the difference bet...,B
3,53567,###Model A\nPrompt: w can I get both of them a...,###Model B\nPrompt: w can I get both of them a...,B
4,53567,###Model A\nPrompt: at is the minimal time to ...,###Model B\nPrompt: at is the minimal time to ...,B
...,...,...,...,...
71293,4294710549,###Model A\nPrompt: e deterrents necessary in ...,###Model B\nPrompt: e deterrents necessary in ...,A
71294,4294710549,###Model A\nPrompt: w realistic is it for toda...,###Model B\nPrompt: w realistic is it for toda...,A
71295,4294710549,###Model A\nPrompt: n you explain concisely wh...,###Model B\nPrompt: n you explain concisely wh...,A
71296,4294899228,"###Model A\nPrompt: If a bait contains 0,0025%...","###Model B\nPrompt: If a bait contains 0,0025%...",B


In [20]:
idx = train.id.unique()
valid_idx = [idx[i] for i in range(len(idx)) if i % 20 == 0]

In [21]:
train.loc[train.id.isin(valid_idx),].to_csv('dataset/random_instruction_valid.csv', index = False)
train.loc[~train.id.isin(valid_idx),].to_csv('dataset/random_instruction_train.csv', index = False)